<a href="https://colab.research.google.com/github/Athulkrishna-S/ML/blob/main/defog_sqlcoder_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SQLCoder-7b-2
Run the cells below to run inference on our text-to-SQL LLM: SQLCoder-7b-2.

⭐️ [Github Repo](https://github.com/defog-ai/sqlcoder)

🤗 [Huggingface Page](https://huggingface.co/defog/sqlcoder-7b-2)

##Setup

In [1]:
!pip install torch transformers bitsandbytes accelerate sqlparse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 28.8 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
torch.cuda.is_available()

True

In [4]:
available_memory = torch.cuda.get_device_properties(0).total_memory

In [5]:
print(available_memory)

15835660288


##Download the Model
Use any model on Colab (or any system with >30GB VRAM on your own machine) to load this in f16. If unavailable, use a GPU with minimum 8GB VRAM to load this in 8bit, or with minimum 5GB of VRAM to load in 4bit.

This step can take around 5 minutes the first time. So please be patient :)

In [6]:
model_name = "defog/sqlcoder-7b-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if available_memory > 15e9:
    # if you have atleast 15GB of GPU memory, run load the model in float16
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        device_map="auto",
        use_cache=True,
    )
else:
    # else, load in 8 bits – this is a bit slower
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        # torch_dtype=torch.float16,
        load_in_8bit=True,
        device_map="auto",
        use_cache=True,
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

##Set the Question & Prompt and Tokenize
Feel free to change the schema in the prompt below to your own schema

In [25]:
prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Remember that revenue is price multiplied by quantity
- Remember that cost is supply_price multiplied by quantity

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY, -- Unique ID for each product
  name VARCHAR(50), -- Name of the product
  price DECIMAL(10,2), -- Price of each unit of the product
  quantity INTEGER  -- Current quantity in stock
);

CREATE TABLE customers (
   customer_id INTEGER PRIMARY KEY, -- Unique ID for each customer
   name VARCHAR(50), -- Name of the customer
   address VARCHAR(100) -- Mailing address of the customer
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY, -- Unique ID for each salesperson
  name VARCHAR(50), -- Name of the salesperson
  region VARCHAR(50) -- Geographic sales region
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, -- Unique ID for each sale
  product_id INTEGER, -- ID of product sold
  customer_id INTEGER,  -- ID of customer who made purchase
  salesperson_id INTEGER, -- ID of salesperson who made the sale
  sale_date DATE, -- Date the sale occurred
  quantity INTEGER -- Quantity of product sold
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY, -- Unique ID for each supplier
  product_id INTEGER, -- Product ID supplied
  supply_price DECIMAL(10,2) -- Unit price charged by supplier
);

-- sales.product_id can be joined with products.product_id
-- sales.customer_id can be joined with customers.customer_id
-- sales.salesperson_id can be joined with salespeople.salesperson_id
-- product_suppliers.product_id can be joined with products.product_id

### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

my prompt

In [49]:
prompt = """### Task
Generate a SQL query to answer {question}

### Instructions



### Database Schema
{schema}

### Answer
Given the database schema, here is the SQL query that answers {question}
[SQL]
"""

##Generate the SQL
This can be excruciatingly slow on a T4 in Colab, and can take 10-20 seconds per query. On faster GPUs, this will take ~1-2 seconds

Ideally, you should use `num_beams`=4 for best results. But because of memory constraints, we will stick to just 1 for now.

In [52]:
import sqlparse

def generate_query(question,schema):
    updated_prompt = prompt.format(question=question,schema=schema)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service

    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

In [ ]:
!pip install datasets sqlparse


my code

In [28]:
from datasets import load_dataset

dataset = load_dataset("b-mc2/sql-create-context")

Generating train split: 0 examples [00:00, ? examples/s]

In [79]:
for example in dataset:
  question=dataset['train'][i]['question']
  schema=dataset['train'][i]['context']
  generated_sql = generate_query(question,schema)
  print('*****************************************************************************')
  print('question : ',question,"\ncontext : ",schema,'\nexpected : ',dataset['train'][i]['answer'])
  print('generated : ',generated_sql)


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 1.06 MiB is free. Process 9376 has 14.74 GiB memory in use. Of the allocated memory 12.97 GiB is allocated by PyTorch, and 70.76 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [59]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')


'en_US.UTF-8'

In [ ]:
import os
os.environ['LC_ALL'] = 'en_US.UTF-8'
!pip install nltk



In [77]:
import tensorflow as tf
import tensorflow_hub as hub

# Load the Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Define the embedding function
def embedding_function(text):
    # Convert the text into embeddings
    embedding = embed([text])
    return embedding

# Example usage:
text = "SELECT * FROM table WHERE condition"
embedding = embedding_function(text)
print("Embedding:", embedding)


Embedding: tf.Tensor(
[[-0.04532933 -0.00242607  0.03021309  0.04214257  0.03761987 -0.04835869
   0.01243483 -0.03893973  0.0142068   0.02543188  0.04674295  0.02214791
   0.0257101   0.03578572  0.04983381  0.08438969 -0.04354007  0.04927631
   0.00981799  0.00161124  0.08057886 -0.0404483   0.04027014  0.03342169
  -0.01497467 -0.03959375  0.05892013  0.04263433 -0.00283313  0.00103688
   0.05299368  0.06241026  0.06931993 -0.0287063  -0.0145372   0.01134406
   0.03783267 -0.03542908 -0.01081947  0.03951863  0.01216279 -0.06019454
  -0.06733496  0.01618617  0.08393665  0.05262113 -0.06426299  0.04437449
   0.01651112 -0.05996999  0.06253711  0.02515741 -0.01137256 -0.00995011
  -0.0400642   0.02409106 -0.03155756 -0.04680029 -0.05422059  0.0127843
  -0.00014484 -0.00360491 -0.04664944 -0.0688036   0.01461421  0.01249456
   0.03878495 -0.04835925  0.02461618 -0.00416005  0.03200137  0.02754652
   0.04426309  0.08172131  0.01721702  0.03245692  0.08431884 -0.02320502
  -0.05382551 -0.

In [78]:
import tensorflow as tf
import numpy as np

# Initialize accuracy metric
accuracy = tf.keras.metrics.Accuracy()

# Iterate through validation examples
count = 0
for example in dataset['train']:
    question = example['question']
    schema = example['context']
    expected_sql = example['answer']
    count += 1

    # Generate SQL query
    generated_sql = generate_query(question, schema)

    # Embed both the expected and generated SQL queries
    expected_embedding = embedding_function(expected_sql)
    generated_embedding = embedding_function(generated_sql)

    # Update accuracy metric
    predicted_label = compare_function(generated_embedding, expected_embedding)
    true_label = 1 if predicted_label == 1 else 0
    accuracy.update_state([true_label], [predicted_label])

    if count == 5:  # For demonstration, limit to 5 examples
        break

# Calculate accuracy
accuracy_result = accuracy.result().numpy()

# Print or log the accuracy
print("Validation Accuracy (using embeddings for both expected and generated SQL):", accuracy_result)


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 1.06 MiB is free. Process 9376 has 14.74 GiB memory in use. Of the allocated memory 12.97 GiB is allocated by PyTorch, and 74.30 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [64]:
import tensorflow as tf
from datasets import load_dataset
import sqlparse
#from nltk.translate.bleu_score import corpus_bleu

# Load dataset for validation


# Initialize metrics
accuracy = tf.keras.metrics.Accuracy()
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()

# Initialize variables for exact match ratio and BLEU score
exact_match_count = 0
bleu_scores = []
count=0
# Iterate through validation examples
for example in dataset['train']:
    count+=1
    question = example['question']
    schema = example['context']
    expected_sql = example['answer']

    # Generate SQL query
    generated_sql = generate_query(question, schema)

    # Format expected SQL
    expected_sql = sqlparse.format(expected_sql, reindent=True)

    # Update accuracy, precision, and recall
    accuracy.update_state([expected_sql], [generated_sql])

    # Update precision and recall
    precision.update_state([expected_sql], [generated_sql])
    recall.update_state([expected_sql], [generated_sql])
    # Update exact match count
    if expected_sql == generated_sql:
        exact_match_count += 1

    # Calculate BLEU score
    reference = [expected_sql.split()]
    candidate = generated_sql.split()
    bleu_score = corpus_bleu(reference, candidate)
    bleu_scores.append(bleu_score)
    if count==10:
      break

# Calculate metrics
accuracy_result = accuracy.result().numpy()
precision_result = precision.result().numpy()
recall_result = recall.result().numpy()
#exact_match_ratio = exact_match_count / len(dataset) * 100
#average_bleu_score = sum(bleu_scores) / len(bleu_scores)

# Calculate F1 score
f1_score = 2 * (precision_result * recall_result) / (precision_result + recall_result)

# Print or log the metrics
print("Validation Accuracy:", accuracy_result)
print("Precision:", precision_result)
print("Recall:", recall_result)
print("F1 Score:", f1_score)
#print("Exact Match Ratio:", exact_match_ratio)
#print("Average BLEU Score:", average_bleu_score)


['### Task\nGenerate a SQL query to answer How many heads of the departments are older than 56 ?\n\n### Instructions\n\n\n\n### Database Schema\nCREATE TABLE head (age INTEGER)\n\n### Answer\nGiven the database schema, here is the SQL query that answers How many heads of the departments are older than 56 ?\n[SQL]\n SELECT COUNT(*) FROM head WHERE age > 56']


UnimplementedError: {{function_node __wrapped__Cast_device_/job:localhost/replica:0/task:0/device:CPU:0}} Cast string to float is not supported [Op:Cast] name: 

In [ ]:
question = "What was our revenue by product in the new york region last month?"
generated_sql = generate_query(question)
print(generated_sql)

Wikkiii sql code.....................................


In [8]:
from torch.utils.data import DataLoader
from tqdm import tqdm
from datasets import load_dataset
import sqlparse

# Load WikiSQL dataset
wikisql_dataset = load_dataset("wikisql")

# Load tokenizer and model
model_name = "defog/sqlcoder-7b-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Generating test split:   0%|          | 0/15878 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8421 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/56355 [00:00<?, ? examples/s]

In [20]:
# Assuming you have already loaded the dataset and named it 'dataset'

# Retrieve an example from the test split
test_example = wikisql_dataset['test'][0]['table']  # Assuming you want the first example in the test split
test_question=wikisql_dataset['test'][0]['question']
# Now 'test_example' contains the example from the test split
print(wikisql_dataset['test'][0]['sql']['human_readable'])
print(test_example)
print(test_question)

SELECT Nationality FROM table WHERE Player = Terrence Ross
{'header': ['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team'], 'page_title': 'Toronto Raptors all-time roster', 'page_id': '', 'types': ['text', 'text', 'text', 'text', 'text', 'text'], 'id': '1-10015132-16', 'section_title': 'R', 'caption': 'R', 'rows': [['Aleksandar Radojević', '25', 'Serbia', 'Center', '1999-2000', 'Barton CC (KS)'], ['Shawn Respert', '31', 'United States', 'Guard', '1997-98', 'Michigan State'], ['Quentin Richardson', 'N/A', 'United States', 'Forward', '2013-present', 'DePaul'], ['Alvin Robertson', '7, 21', 'United States', 'Guard', '1995-96', 'Arkansas'], ['Carlos Rogers', '33, 34', 'United States', 'Forward-Center', '1995-98', 'Tennessee State'], ['Roy Rogers', '9', 'United States', 'Forward', '1998', 'Alabama'], ['Jalen Rose', '5', 'United States', 'Guard-Forward', '2003-06', 'Michigan'], ['Terrence Ross', '31', 'United States', 'Guard', '2012-present', 'Washington']], 'n

In [65]:
m = tf.keras.metrics.Accuracy()
m.update_state([[0], [2], [3], [4]], [[0], [2], [3], [4]])
m.result().numpy()


1.0